# BEE 4750 Lab 4: Simulation-Optimization

**Name**: Brooke Paykin

**ID**: bsp67

> **Due Date**
>
> Friday, 11/17/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [12]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/github-classroom/BEE4750-FA23/lab04-bsp67`


In [13]:
using Random # for random seeds
using Distributions # statistical distribution interface
using Roots # find zeros of functions
using Metaheuristics # search algorithms
using Plots # plotting

## Overview

In this lab, you will experiment with simulation-optimization with the
shallow lake problem. The goal of this experimentation is to get an
understanding of how to work with simulation-optimization methods and
the impact of some choices involved in using these methods.

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Due to ongoing economic activity, a town emits phosphorous into a
shallow lake (with a concentration of $a_t$), which also receives
non-point source runoff (concentration $y_t$) from the surrounding area.
The concentration of the lake at time $t+1$ is given by
$$X_{t+1} = X_t + a_t + y_t + \frac{X_t^q}{1+X_t^q} - bX_t,$$

where:

| Parameter | Value                                                |
|:---------:|:-----------------------------------------------------|
|   $a_t$   | point-source phosphorous concentration from the town |
|   $y_t$   | non-point-source phosphorous concentration           |
|    $q$    | rate at which phosphorous is recycled from sediment  |
|    $b$    | rate at which phosphorous leaves the lake            |

and $X_0 = 0$, $y_t \sim LogNormal(\log(0.03), 0.25)$, $q=2.5$, and
$b=0.4$.

The goal of the optimization is to maximize the town’s average
phosphorous concentration releases (as a proxy of economic activity):
$\max \sum_{t=1}^T a_t / T$ over a 100-year period. We have decided
(initially) that an acceptable solution is one which will result in the
lake eutrophying no more than 10% of the time.

The non-point source samples can be sampled using the following code
block:

In [14]:
Random.seed!(1)

T = 100 # length of simualtion
n_samples = 1_000 # replace with number of samples if you experiment

P_distribution = LogNormal(log(0.03), 0.25)
y = rand(P_distribution, (T, n_samples)) # sample a T x n_samples matrix

100×1000 Matrix{Float64}:
 0.0294753  0.0459864  0.023513   …  0.0259183  0.0260934  0.0284652
 0.034263   0.0222782  0.0459188     0.0288482  0.0480454  0.0531018
 0.0245199  0.0296271  0.0445619     0.0246404  0.0250734  0.0304308
 0.055448   0.0312     0.0228208     0.0298609  0.0428105  0.0256198
 0.0401417  0.024978   0.0458244     0.0228935  0.0286062  0.0238694
 0.0320754  0.021759   0.0471452  …  0.0472771  0.0187508  0.0306753
 0.0464641  0.0416385  0.0246833     0.0382252  0.0288505  0.0226561
 0.0244027  0.0432707  0.0341214     0.0238988  0.0427204  0.0316143
 0.0231156  0.0279197  0.0217747     0.0231772  0.0335662  0.0324465
 0.0276303  0.0305858  0.0440326     0.0289394  0.0312328  0.0173388
 ⋮                                ⋱                        
 0.025665   0.0341366  0.0274747     0.0283546  0.0458031  0.0277959
 0.0405629  0.0421121  0.0252557     0.0450377  0.0284411  0.0206434
 0.0228445  0.0223746  0.0210942     0.0442834  0.0337672  0.0287835
 0.0252604  0.046

We write the lake model as a function:

In [15]:
# lake function model
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   series of lake phosphorous concentrations
function lake(a, y, q, b, T)
    X = zeros(T+1, size(y, 2))
    # calculate states
    for t = 1:T
        X[t+1, :] = X[t, :] .+ a[t] .+ y[t, :] .+ (X[t, :].^q./(1 .+ X[t, :].^q)) .- b.*X[t, :]
    end
    return X
end

lake (generic function with 1 method)

However, this isn’t sufficient on its own! `Metaheuristics.jl` (and most
simulation-optimization packages) require the use of a *wrapper*
function, which accepts as inputs both parameters to be optimized (in
this case, point-source releases `a`) and parameters which will be fixed
(the others; see below for how to incorporate these into the syntax) and
returns the required information for the optimization procedure.

`Metaheuristics.jl` wants its optimizing wrapper function to return (in
order):

-   the objective(s) (in this case, the mean of `a`, $\sum_t a_t / T$),
-   a vector of the degrees to which the solution fails to achieve any
    inequality constraints (positive values indicate a larger failure,
    values below zero are considered acceptable)
-   a vector of the degrees to which the solution fails to achieve any
    equality constraints (only values of zero indicate success), which
    in this case is not relevant, so we just return `[0.0]`.

In [16]:
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.1] # replace 0.1 if you experiment with the failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

To optimize using DE (differential evolution), use the following syntax:

``` julia
results = optimize(f, bounds, DE(options=Options(f_calls_limit=max_evals)))
```

where `bounds` is a `Matrix` of lower bounds (first row) and upper
bounds (last row), and `max_evals` is an integer for the maximum number
of evaluations.

-   For example, to set bounds for all decision variables between 0 and
    0.5, you can use

``` julia
bounds = [zeros(T) 0.5ones(T)]'
```

-   Increasing `max_evals` can help you find a better solution, but at a
    larger computational expense.
-   You can use an anonymous function to fix values for non-optimized
    parameters, *e.g.*

``` julia
y = ...
q = ...
b = ...
T = ...
Xcrit = ...
results = optimize(a -> lake_opt(a, y, q, b, t, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))
```

Then to get the approximated minimum value:

``` julia
fx = minimum(result)
```

and the approximated minimizing value:

``` julia
x = minimizer(result)
```

The last piece is to get the critical value (to identify failures),
which we can do using `Roots.jl`, which finds zeros of functions:

In [17]:
#phosphorus recyling rate
q=2.5

#phospohorus leaving rate
b=0.4 

# define a function whose zeros are the critical values
P_flux(x) = (x^q/(1+x^q)) - b*x
# use Roots.find_zero() to find the non-eutrophication and non-zero critical value; we know from visual inspection in class that this is bounded between 0.1 and 1.5.
Xcrit = find_zero(P_flux, (0.1, 1.5))

0.6678778690448219

## Problems

### Problem 1 (2 points)

Using the default setup above, find the approximate optimum value. What
is the value of the objective function, and how many failures (you can
evaluate the `lake` function using your solution to find how many
end-values are above the critical value).


In [29]:
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 1000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


According to my code number of failures was 1000. The number of simulations run (max_evals) was also 1000. This means that there was a 100% failure rate, meaning that all simulations caused there to be eutrophication.

According to the code, the optimal objective value is approximately 0.2105. 

### Problem 2 (5 points)
Feel free to experiment with some of the degrees of freedom in finding
the optimum value. For example:

-   What failure probability are you using to characterize acceptable
    solutions?
-   How many Monte Carlo samples are you using?
-   What bounds are you searching over for the releases?
-   How many function evaluations are you using for the search?
-   What is the impact of different [`Metaheuristics.jl`
    algorithms](https://docs.juliahub.com/Metaheuristics/aJ70z/3.2.12/algorithms/)?

Note that you might want to modify some of these together: for example,
lower acceptable failure probabilities often require more function
evaluations to find acceptable values, and more Monte Carlo samples
increase computational expense, so fewer function evaluations may be
completed in the same time.

Provide a description of what you’ve modified and why. What was the new
solution that you found? Does it satisfy the constraints?

EXPERIMENT #1: Exploring failure probability

In problem 1, the failure probabily is 10%. 

I will run an experiment code with a lower failure probabily by changing the failure probability to 5%. Since this is defined in the lake function, I will first need to modify the lake function and manipulate the failconst  to be Pexceed - 0.05 instead of Pexceed - 0.1. Since I lowered the failure probability and I know that lower acceptable failure probabilities often require more function evaluations to find acceptable values, I will also increase max_evals from 1000 to 2000.

Next, I will run an experiment code with a higher failure probabily by changing the failure probability to 15%. Since this is defined in the lake function, I will first need to modify the lake function and manipulate the failconst  to be Pexceed - 0.15 instead of Pexceed - 0.1. For experiments where I have increased the failure porbability, I will not increase my number of function evaluations as it is lower acceptable probabilities that often require more function evalautions, not higher probabilities. As such, max_evals will be set to 1000. 

Next,  I will run an experiment code with an even higher failure probabily by changing the failure probability to 50%. Since this is defined in the lake function, I will first need to modify the lake function and manipulate the failconst  to be Pexceed - 0.50 instead of Pexceed - 0.1. For experiments where I have increased the failure porbability, I will not increase my number of function evaluations as it is lower acceptable probabilities that often require more function evalautions, not higher probabilities. As such, max_evals will be set to 1000. 



In [42]:
#EXPLORING FAILURE PROBABILY BY REDUCING IT
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.05] # replaced 0.1 with 0.05 to experiment with the failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [43]:
#EXPLORING FAILURE PROBABILY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 2000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.20390011684353343
Number of failures: 1000


Now I will increase the failure probability to 15%

In [37]:
#EXPLORING FAILURE PROBABILY BY INCREASING IT
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.15] # replaced 0.1 with 0.15 to experiment with the failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [38]:
#EXPLORING FAILURE PROBABILY BY INCREASING IT
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 1000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


In [39]:
#EXPLORING FAILURE PROBABILY BY INCREASING IT
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.50] # replaced 0.1 with 0.50 to experiment with the failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [40]:
#EXPLORING FAILURE PROBABILY BY INCREASING IT
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 1000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


I found that when I decreased the failure probability (I tested 0.05) and changed my number of evaluations to 2000, my optimal objective value changed slightly. It became 0.20390011684353343
instead of 0.21058759608000296. Interestly, the number of failures remained 1000. This means that there was a 50% fail rate and eutrophication resulted in 50% of my simualtions. 


When I increased the failure probability (I tested 0.15 and 0.50) and kept the number of evaluations at 1000, my optimal objective value did not change from problem 1 nor did it change my number of failures from problem 1. In other words, the optimal objective value was  0.21058759608000296 and the number of failures remained 1000. This means that there was a 100% fail rate and eutrophication resulted in 100% of my simualtions. 


I then decided that I wanted to test what would happen if I only changed the number of evalutions to determine. I decided to keep the failure probability at 10% but change the my number of evaluations to 2000. Please see below for the experiment and results.

In [44]:
#EXPLORING NUMBER OF EVAULATIONS ONLY
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.10] # 0.10 failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [45]:
#EXPLORING NUMBER OF EVALUATIONS ONLY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 2000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.20390011684353343
Number of failures: 1000


Note that whether the max_evals = 2000 or 1000, there are still reported to be 1000 failures. 

I found that whether the failure probability was 5% or 10%, it did not affect the optimal solution. However, when the sample size was increased, the optimal solution changed. Indeed, a failure probability of 5% and max_evals = 2000 yields an optimal value of approximately 0.2039 AND failure probability of 10% and max_evals = 2000 yields an optimal value of approximately 0.2039. This implies that it was the max_evals that effected the optimal value, not the failure probability. To test this, I decided to run experiments with increased failure probability and increased max_evals. If the results reamin 0.2039 and 1000, then the implication still appears to be correct.

In [46]:
#increase NUMBER OF EVAULATIONS to 2000  + increase failure probability to 0.5
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.50] # 0.50 failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [47]:
#EXPLORING NUMBER OF EVALUATIONS ONLY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 2000

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.20390011684353343
Number of failures: 1000


Even with a failure probability of 0.50, when the max_evals was set to 2000 it yielded the answer Optimal objective value: 0.20390011684353343 and Number of failures: 1000. This once again affirms that only max_evals are effecting the optimal objective value and that the failure probability is not affecting the optimal objective failure. 

I decided to experiment with max evals being decreased next and reset the failure probabiliyt to 10%. I am first testing max_evals = 750, then 500, then 150

In [48]:
#decrease NUMBER OF EVAULATIONS to 2000  + reset failure probability to 0.1
# function producing optimization outputs
# inputs:
#   a: vector of point-source releases (to be optimized)
#   y: randomly-sampled non-point sources
#   q: lake phosphorous recycling rate
#   b: phosphorous outflow rate
# 
# returns:
#   - objective: mean value of point-source releases
#   - inequality constraint failure vector
#   - equality constraint failure vector (in this case, always [0.0])
function lake_opt(a, y, q, b, T, Xcrit)
    X = lake(a, y, q, b, T)
    # calculate exceedance of critical value
    Pexceed = sum(X[T+1, :] .> Xcrit) / size(X, 2)
    failconst = [Pexceed - 0.10] # 0.10 failure probability
    return mean(a), failconst, [0.0]
end

lake_opt (generic function with 1 method)

In [49]:
#EXPLORING NUMBER OF EVALUATIONS ONLY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 750

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


In [50]:
#EXPLORING NUMBER OF EVALUATIONS ONLY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 500

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

#get the critical value (to identify failures), which we can do using Roots.jl, which finds zeros of function
#Xcrit = find_zero(P_flux, (0.1, 1.5)) #redunant bc previously defined, but i wanted to write it here

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


In [51]:
#EXPLORING NUMBER OF EVALUATIONS ONLY
# parameters y, q, b, T, Xcrit were previously defined

Random.seed!(1)

#set bounds for all decision variables between 0 and 0.5
bounds = [zeros(T) 0.5*ones(T)]'

#set # of simulations
max_evals = 150

results = optimize(a -> lake_opt(a, y, q, b, T, Xcrit), bounds, DE(options=Options(f_calls_limit=max_evals)))

# get approximated minimum value
fx = minimum(results)

#get approximated minimizing value
x = minimizer(results)

# Evaluate the lake function using the solution to count failures
X_opt = lake(x, y, q, b, T)
num_failures = sum(X_opt[T+1, :] .> Xcrit)

# Display the results
println("Optimal objective value: $fx")
println("Number of failures: $num_failures")


Optimal objective value: 0.21058759608000296
Number of failures: 1000


When max_evals was 150, 500, 750, and 1000 (and % failure was held constant at 10%), it yielded the same results: Optimal objective value: 0.21058759608000296 and Number of failures: 1000. Note that the number that max_evals is set to is the number of monte carlo simulations.

This is confusing because I thought if I put max_evals at 150, then a 100% failure rate would mean teh number of failures is 150. Somehow though, the number of failures exceeds the number of trials for 150, 500, and 750 which does not make sense. This implies it does not satisfy the constraints.

### Problem 3 (3 points)

What did you learn about the use of these methods? Compare with your
experience with linear programming from earlier in the semester.

## References

Put any consulted sources here, including classmates you worked with/who
helped you.